In [1]:
import numpy as np
import pyrem as pr

In [2]:
a = np.random.rand(32)
b = np.ones(32)

In [3]:
a

array([0.12961047, 0.43948143, 0.33084862, 0.38431153, 0.67813697,
       0.74755739, 0.51931955, 0.05536134, 0.97119633, 0.5035879 ,
       0.16704713, 0.94710526, 0.2431946 , 0.68970834, 0.12166739,
       0.04462404, 0.08808697, 0.4590722 , 0.5134365 , 0.17800184,
       0.6206704 , 0.13942664, 0.30266957, 0.46036043, 0.63696304,
       0.69136591, 0.13585384, 0.29425137, 0.6812238 , 0.4297085 ,
       0.42880942, 0.79781275])

In [4]:
b

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [5]:
def calculateStandardDeviation(a):
    return np.std(a)
calculateStandardDeviation(a)

0.25825415594352313

In [8]:
c = np.random.normal(size=int(1e6))

In [11]:
#suppose EEG data mean very close to 0, from http://gilestrolab.github.io/pyrem/_modules/pyrem/univariate.html
def calculateHjorth(a):
    first_deriv = np.diff(a)
    second_deriv = np.diff(a,2)
    var_zero = np.mean(a ** 2)
    var_d1 = np.mean(first_deriv ** 2)
    var_d2 = np.mean(second_deriv ** 2)
    activity = var_zero
    morbidity = np.sqrt(var_d1 / var_zero)
    complexity = np.sqrt(var_d2 / var_d1) / morbidity
    return activity, morbidity, complexity
calculateHjorth(c)

(0.9991286420543838, 1.4141271837868732, 1.2250726442470694)

In [ ]:
# confused by term 'slide window' in the paper
def calculateMMD(a):
    return np.max(a) - np.min(a) # only one dimention

In [22]:
#Petrosian Fractal Dimension, complexity
def calculatePFD(a):
    N = len(a)
    diff = np.ediff1d(a)
    sign_change = (diff[1:-1] * diff[0:-2] < 0)
    M = sign_change.sum()
    return np.log10(N) / (np.log10(N) + np.log10(N / (N + 0.4 * M)))
calculatePFD(a)

In [34]:
#using Katz fractal dimension instead of Normalized Line Length (NLL)
def calculateKFD(a):
    distance = np.abs(np.ediff1d(a))
    LL = distance.sum() #'line length'
    LL_normalized = np.log10(np.divide(LL, distance.mean())) #original paper uses constant hyperparameter M=1
    aux_d = a - a[0]
    d = np.max(np.abs(aux_d[1:]))
    return np.divide(LL_normalized, np.add(LL_normalized, np.log10(np.divide(d, LL))))

print(calculateNLL(c))
print(calculateNLL_(c))

8.20795990200413
403628.9763900714


In [42]:
#Hurst Exponent: failed to understand the equation

In [45]:
#Log Root Sum of Sequential Variations
#measure the sequential variations
def calculateLRSSV(a):
    diff = np.ediff1d(a)
    return np.log10(np.sqrt(np.sum(diff**2)))

calculateLRSSV(b)

/Users/wzy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log10
  after removing the cwd from sys.path.


-inf

In [50]:
#Normalized Spectral Entropy, https://raphaelvallat.com/entropy/build/html/_modules/entropy/entropy.html
from scipy.signal import periodogram
def calculateSE(a):
    _, psd = periodogram(a, 100)  #fft transform
    psd_norm = np.divide(psd, psd.sum()) #power spectral density, measure of signal's power content versus frequency
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    se /= np.log2(psd_norm.size)
    return se
calculateSE(a)

0.8196842663563354

In [51]:
#Rényi entropy (RE) a measure of the entropy of the distribution P = (p1, p2,..., pn)
def calculateRE(a, m=2):
    return 1/(1-m) * np.log2(np.sum(a**m))
calculateRE(b)

-5.0

In [60]:
from scipy.special import digamma
#Kraskov entropy (KE) an estimate for Shannon entropy using N samples of an m-dimensional random vector x
#very confused???
def calculateKE(a):
    N = len(a)
    k = 5 #what should be k?
    sum_ = 0
    for i in range(len(a)):
        kth_nearest = sorted(sorted(a, key = lambda n: abs(a[i]-n))[:k])[-1]
        ri = np.abs(kth_nearest - a[i])
        if ri!=0:
            sum_ += np.log(2*ri)
    
    
    return digamma(N) - digamma(k) + 1/N * sum_
calculateKE(a)

0.037436658476834195
0.020878997808118527
0.05346290778783391
0.055169901588860815
0.01322894126230445
0.05025536226430416
0.0
0.074249136844174
0.0
0.015731645229023927
0.010954705297026246
0.024091074154096948
0.05947496446681155
0.0016575692093764305
0.017759241671262305
0.0849864348951247
0.04152350698679652
0.0012882260257599532
0.005883049300184351
0.0
0.0690379441417237
0.027620497188938553
0.08164195866019086
0.0
0.052745296659490126
0.0
0.03119328995360071
0.09006015651350452
0.010142105251218503
0.0306519243265293
0.03155100548333911
0.0


-0.4647181237371938